In [ ]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

### Optionally: 
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

In [ ]:
import json
from langchain.schema import Document

with open('data/reddit-data.json', 'r') as f:
    reddit_data = json.load(f)


docs = [
    Document(
        page_content=f"{item['title']}\n\n{item['selftext']}",
        metadata={
            "subreddit": item["subreddit"],
            "upvotes": item["upvotes"],
            "url": item["url"],
            "timestamp": item["created_utc"],
        },
    )
    for item in reddit_data
]

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embedding_model)
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, State
from langchain_openai import ChatOpenAI
from typing import TypedDict

# Define graph state
class ToolkitState(TypedDict):
    user_goal: str
    category_plan: list[str]
    recommendations: dict[str, str]
def rag_node(state: ToolkitState) -> ToolkitState:
        # Just take top doc's content for demo
        if docs:
            updated_recs[category] = docs[0].page_content
        else:
            updated_recs[category] = "No recommendation found."
    return {
        **state,
        "recommendations": updated_recs
    }